In [9]:
#Downloads
nltk.download('punkt')
nltk.download("stopwords")
SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/louisbove84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/louisbove84/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [42]:
import pandas as pd
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from nltk import pos_tag
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Supervised

In [80]:
#Import Twitter and Reddit DataFrames
df_twitter = pd.read_csv('twitter.csv')

#Normalize the likes/retweets/follows

In [71]:
# Function to generate n-grams from sentences.
def extract_ngrams(text_lst, num):
    n_grams = ngrams(nltk.word_tokenize(' '.join(text_lst)), num)
    return [ ' '.join(grams) for grams in n_grams]

#Tokenize, Lowercase, and Filter Stopwords for Column in DataFrame
def tokenize(df, col_name):
    #Pull down stopwords and punctuation
    punctuation_ = set(string.punctuation)
    stopwords_ = set(stopwords.words('english'))
    
    #Use Snowball Stemmer for stemming
    stemmer_snowball = SnowballStemmer('english') 
    
    #Modify text
    df[col_name] = df[col_name].apply(lambda row: [w.split('://')[0] for w in row]) #Remove links 
    df[col_name] = df[col_name].apply(lambda row: word_tokenize(row)) #Tokenize (separate words)
    df[col_name] = df[col_name].apply(lambda row: [w.lower() for w in row]) #Lowercase
    df[col_name] = df[col_name].apply(lambda row: [w for w in row if not w in punctuation_]) #Remove Stopwords
    df[col_name] = df[col_name].apply(lambda row: [w for w in row if not w in stopwords_]) #Remove Punctuation
    df[col_name] = df[col_name].apply(lambda row: [stemmer_snowball.stem(w) for w in row]) #Stemming (taking words back to the basics)
    df[col_name] = df[col_name].apply(lambda row: extract_ngrams(row,1) + extract_ngrams(row,2)) #Add 2-Grams 
    df[col_name] = df[col_name].apply(lambda row: pos_tag(row))#Tag speech with noun/adj/verb/etc.
    
    
    return df

In [81]:
#Make all words lowercase
df_twitter['tokenized'] = df_twitter['text'].apply(lambda row: row.split('://')[0]) 

#Tokenize tweets
df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: word_tokenize(row)) 

#Make all words lowercase
df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: [word.lower() for word in row]) 

#Filter stopwords
stopwords_ = set(stopwords.words('english'))
punctuation_ = set(string.punctuation)
df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: [w for w in row if not w in punctuation_]) 

#Stemming
stemmer_snowball = SnowballStemmer('english')
df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: [stemmer_snowball.stem(word) for word in row]) 

#Add N-Grams
#df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: extract_ngrams(row, 1) + extract_ngrams(row, 2)) 

#Part-of-Speech tagging (create tuple of word and noun/verb/adj/etc.)
#df_twitter['tokenized'] = df_twitter['tokenized'].apply(lambda row: pos_tag(row)) 

In [92]:
#Vectorize the bag-of-words using TFIDF
def dummy_fun(doc):
    return doc

vectorizer = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

vectors = vectorizer.fit_transform(df_twitter['tokenized'])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_twitter_tfidf = pd.DataFrame(denselist, columns = feature_names)

In [93]:
df_twitter_tfidf.shape

(2640, 6211)

In [101]:
df_twitter_tfidf.iloc[4:8]

,'','d,'m,'s,+100,--,-/,-100,-3,-doctor,...,🤷‍♂️,🥧,🥧we,🥰,🥰🐀,🥰🚀,🥳,🥳🎆,🥺,🦸🏻
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.000000,0.149820,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.137353,0.091855,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Unsupervised